In [1]:
class Card:
    SUITS = ['Clubs', 'Diamonds', 'Hearts', 'Spades', 'Joker']
    RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace', 'Joker']

    def __init__(self, rank, suit):
        if suit not in self.SUITS:
            raise ValueError(f"Invalid suit: {suit}")
        if suit == 'Joker':
            if rank != 'Joker':
                raise ValueError("Joker suit must have Joker rank")
        elif rank not in self.RANKS[:-1]:
            raise ValueError(f"Invalid rank: {rank}")
        
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return "Joker" if self.suit == 'Joker' else f"{self.rank} of {self.suit}"

    def __repr__(self):
        return f"Card(rank='{self.rank}', suit='{self.suit}')"

    def __eq__(self, other):
        return isinstance(other, Card) and self.rank == other.rank and self.suit == other.suit

    def __hash__(self):
        return hash((self.rank, self.suit))

    def __lt__(self, other):
        if not isinstance(other, Card):
            return NotImplemented
        suit_order = self.SUITS
        rank_order = self.RANKS
        if self.suit != other.suit:
            return suit_order.index(self.suit) < suit_order.index(other.suit)
        return rank_order.index(self.rank) < rank_order.index(other.rank)

    def is_wild(self):
        """Returns True if the card is a Joker or a 2 (wild cards in Canasta)."""
        return self.suit == 'Joker' or self.rank == '2'

    def is_red_three(self):
        """Returns True if the card is a red 3 (Diamonds or Hearts)."""
        return self.rank == '3' and self.suit in ['Hearts', 'Diamonds']

    def is_black_three(self):
        """Returns True if the card is a black 3 (Spades or Clubs)."""
        return self.rank == '3' and self.suit in ['Spades', 'Clubs']


In [2]:
# Example usage of all functions in the Card class

# Create some Card instances
card1 = Card('Jack', 'Clubs')
card2 = Card('King', 'Diamonds')
card3 = Card('Joker', 'Joker')

# __str__ method
print(str(card1))  # Output: Jack of Clubs
print(str(card3))  # Output: Joker

# __repr__ method
print(repr(card2))  # Output: Card(rank='King', suit='Diamonds')

# __eq__ method
print(card1 == Card('Jack', 'Clubs'))  # Output: True
print(card1 == card2)                  # Output: False

# __hash__ method (using Card as dictionary keys)
card_dict = {card1: "First card", card2: "Second card"}
print(card_dict[card1])  # Output: First card

# Attempt to create an invalid card (should raise ValueError)
try:
    invalid_card = Card('11', 'Clubs')
except ValueError as e:
    print(e)  # Output: Invalid rank: 11

try:
    invalid_joker = Card('Ace', 'Joker')
except ValueError as e:
    print(e)  # Output: Joker suit must have Joker rank

Jack of Clubs
Joker
Card(rank='King', suit='Diamonds')
True
False
First card
Invalid rank: 11
Joker suit must have Joker rank


In [3]:
import random

class Deck:
    def __init__(self, num_decks=2):
        """Creates the draw pile with num_decks of standard 54-card decks (including Jokers)."""
        self.cards = []
        for _ in range(num_decks):
            for suit in Card.SUITS:
                if suit == 'Joker':
                    # Add Jokers: two per deck
                    self.cards.append(Card(rank='Joker', suit='Joker'))
                    self.cards.append(Card(rank='Joker', suit='Joker'))
                else:
                    for rank in Card.RANKS[:-1]:  # Exclude the Joker rank here
                        self.cards.append(Card(rank=rank, suit=suit))
        self.shuffle()

    def shuffle(self):
        """Shuffles the deck."""
        random.shuffle(self.cards)

    def draw(self, count=1):
        """Draws one or more cards from the top of the deck."""
        if count > len(self.cards):
            raise ValueError("Not enough cards left to draw")
        drawn = self.cards[:count]
        self.cards = self.cards[count:]
        return drawn if count > 1 else drawn[0]

    def add_cards(self, cards):
        """Adds cards back into the deck (e.g., when reshuffling discard pile)."""
        if isinstance(cards, Card):
            self.cards.append(cards)
        else:
            self.cards.extend(cards)

    def is_empty(self):
        """Returns True if the deck is empty."""
        return len(self.cards) == 0

    def __len__(self):
        return len(self.cards)

    def __str__(self):
        return f"Deck with {len(self.cards)} cards"

    def __repr__(self):
        return f"Deck({self.cards})"


In [4]:
deck = Deck()
print(deck)

# Draw 1 card
card = deck.draw()
print("Drew:", card)

# Draw 5 more cards
hand = deck.draw(5)
print("Hand:", hand)

# Check remaining cards
print(f"Cards left: {len(deck)}")


Deck with 108 cards
Drew: 6 of Hearts
Hand: [Card(rank='4', suit='Hearts'), Card(rank='6', suit='Diamonds'), Card(rank='4', suit='Spades'), Card(rank='8', suit='Diamonds'), Card(rank='6', suit='Clubs')]
Cards left: 102


In [5]:
class Hand:
    def __init__(self):
        """Creates an empty hand."""
        self.cards = []

    def add_cards(self, new_cards):
        """Adds one or more cards to the hand."""
        if isinstance(new_cards, Card):
            self.cards.append(new_cards)
        else:
            self.cards.extend(new_cards)

    def remove_card(self, card):
        """Removes a specific card from the hand."""
        if card in self.cards:
            self.cards.remove(card)
            return card
        else:
            raise ValueError(f"Card {card} not found in hand")

    def discard_card(self, card):
        """Alias for remove_card; could later trigger discard pile logic."""
        return self.remove_card(card)

    def has_card(self, rank, suit=None):
        """Checks if the hand contains a card by rank (and optionally suit)."""
        return any(c.rank == rank and (suit is None or c.suit == suit) for c in self.cards)

    def get_all_cards(self):
        """Returns a copy of all cards in the hand."""
        return list(self.cards)

    def sort_hand(self):
        """Sorts the hand using the Card class's comparison operators."""
        self.cards.sort()

    def __len__(self):
        return len(self.cards)

    def __str__(self):
        return ', '.join(str(card) for card in self.cards)

    def __repr__(self):
        return f"Hand({self.cards})"


In [7]:
deck = Deck()
hand = Hand()

# Draw 7 cards to start
hand.add_cards(deck.draw(7))
print("Initial hand:", hand)

# Remove a card
some_card = hand.get_all_cards()[0]
hand.remove_card(some_card)
print("After removing a card:", hand)

# Sort the hand
hand.sort_hand()
print("Sorted hand:", hand)


Initial hand: 8 of Hearts, Queen of Hearts, Queen of Clubs, 9 of Spades, Joker, Ace of Hearts, Jack of Hearts
After removing a card: Queen of Hearts, Queen of Clubs, 9 of Spades, Joker, Ace of Hearts, Jack of Hearts
Sorted hand: Queen of Clubs, Jack of Hearts, Queen of Hearts, Ace of Hearts, 9 of Spades, Joker


In [8]:
class Player:
    def __init__(self, name):
        """Creates a player with a name and an empty hand."""
        self.name = name
        self.hand = Hand()
        self.melds = []  # List of lists of Cards (each meld is a list of cards)
        self.has_gone_out = False

    def draw_from_deck(self, deck, count=1):
        """Draw cards from the deck and add them to the hand."""
        drawn = deck.draw(count)
        self.hand.add_cards(drawn)

    def discard(self, card, discard_pile):
        """Discard a card from hand to the discard pile."""
        discarded = self.hand.discard_card(card)
        discard_pile.add_card(discarded)
        return discarded

    def form_meld(self, cards):
        """Forms a meld from specified cards in the hand."""
        # Remove the cards from the hand
        for card in cards:
            self.hand.remove_card(card)
        self.melds.append(cards)

    def show_hand(self):
        """Returns the cards in the hand as a string."""
        return str(self.hand)

    def show_melds(self):
        """Returns a string representation of the player's melds."""
        return [', '.join(str(card) for card in meld) for meld in self.melds]

    def go_out(self):
        """Marks the player as having gone out."""
        self.has_gone_out = True

    def __str__(self):
        return f"Player {self.name}"

    def __repr__(self):
        return f"Player(name='{self.name}')"


In [10]:
class DiscardPile:
    def __init__(self):
        """Creates an empty discard pile."""
        self.cards = []
        self.is_frozen = False  # Optional: whether the pile is frozen (wilds or jokers on top)

    def add_card(self, card):
        """Adds a card to the top of the discard pile and updates frozen status."""
        self.cards.append(card)
        if card.is_wild() or card.is_red_three():
            self.freeze()

    def top_card(self):
        """Returns the top card of the discard pile without removing it."""
        if not self.cards:
            return None
        return self.cards[-1]

    def take_pile(self):
        """Removes and returns all cards from the pile (used when picking up the pile)."""
        taken_cards = list(self.cards)
        self.cards.clear()
        self.unfreeze()
        return taken_cards

    def freeze(self):
        """Freezes the pile, making it harder to pick up (as per Canasta rules)."""
        self.is_frozen = True

    def unfreeze(self):
        """Unfreezes the pile."""
        self.is_frozen = False

    def is_empty(self):
        return len(self.cards) == 0

    def __len__(self):
        return len(self.cards)

    def __str__(self):
        top = self.top_card()
        return f"DiscardPile(top={top}, frozen={self.is_frozen}, size={len(self.cards)})"

    def __repr__(self):
        return f"DiscardPile(cards={self.cards})"


In [12]:
deck = Deck()
discard_pile = DiscardPile()  # We'll build this next
player = Player(name="Lobke")

# Initial draw
player.draw_from_deck(deck, count=11)
print(f"{player.name}'s hand: {player.show_hand()}")

# Discard a random card
card_to_discard = player.hand.get_all_cards()[0]
player.discard(card_to_discard, discard_pile)
print(f"{player.name} discarded {card_to_discard}")

# Show updated hand
print(f"Updated hand: {player.show_hand()}")


Lobke's hand: 10 of Clubs, 9 of Clubs, 5 of Spades, Queen of Spades, 7 of Hearts, 4 of Spades, Ace of Hearts, 10 of Hearts, 2 of Hearts, 9 of Diamonds, Ace of Clubs
Lobke discarded 10 of Clubs
Updated hand: 9 of Clubs, 5 of Spades, Queen of Spades, 7 of Hearts, 4 of Spades, Ace of Hearts, 10 of Hearts, 2 of Hearts, 9 of Diamonds, Ace of Clubs


In [13]:
discard_pile = DiscardPile()

# Add a card
discard_pile.add_card(Card(rank='5', suit='Hearts'))
print(discard_pile)

# Add a Joker (freezes the pile)
discard_pile.add_card(Card(rank='Joker', suit='Joker'))
print(discard_pile)

# Take the pile
pile_cards = discard_pile.take_pile()
print(f"Took discard pile: {pile_cards}")
print(discard_pile)


DiscardPile(top=5 of Hearts, frozen=False, size=1)
DiscardPile(top=Joker, frozen=True, size=2)
Took discard pile: [Card(rank='5', suit='Hearts'), Card(rank='Joker', suit='Joker')]
DiscardPile(top=None, frozen=False, size=0)


In [14]:
class Game:
    def __init__(self, player_names):
        """Initializes the game with 4 players (two teams)."""
        if len(player_names) != 4:
            raise ValueError("Canasta requires exactly 4 players.")

        # Create players
        self.players = [Player(name) for name in player_names]

        # Divide players into two teams
        self.teams = {
            "Team A": [self.players[0], self.players[2]],
            "Team B": [self.players[1], self.players[3]]
        }

        # Create deck and discard pile
        self.deck = Deck()
        self.discard_pile = DiscardPile()

        # Deal initial hands
        self.deal_initial_hands()

        # Track whose turn it is
        self.current_player_index = 0

    def deal_initial_hands(self, cards_per_player=11):
        """Deals the starting hand to all players."""
        for player in self.players:
            player.draw_from_deck(self.deck, cards_per_player)

    def next_player(self):
        """Advances to the next player's turn."""
        self.current_player_index = (self.current_player_index + 1) % len(self.players)

    def current_player(self):
        """Returns the player whose turn it is."""
        return self.players[self.current_player_index]

    def play_turn(self):
        """Simulates a simple turn for the current player."""
        player = self.current_player()
        print(f"\n--- {player.name}'s turn ---")

        # 1. Draw a card
        drawn_card = self.deck.draw()
        player.hand.add_cards(drawn_card)
        print(f"{player.name} drew {drawn_card}")

        # 2. TODO: Add meld decision logic here

        # 3. Discard the first card in hand for simplicity
        discard_card = player.hand.get_all_cards()[0]
        player.discard(discard_card, self.discard_pile)
        print(f"{player.name} discarded {discard_card}")

        # Advance to next player
        self.next_player()

    def play_round(self):
        """Plays one full round (each player takes one turn)."""
        for _ in self.players:
            self.play_turn()

    def show_game_state(self):
        """Prints a summary of the current game state."""
        print("\n=== Game State ===")
        for player in self.players:
            print(f"{player.name}: {player.show_hand()}")
        print(self.discard_pile)

    def is_game_over(self):
        """Checks if any player has gone out (simplified end condition)."""
        return any(player.has_gone_out for player in self.players)


In [15]:
game = Game(["Ruud", "Mariska", "Menno", "Lobke"])

# Play 3 rounds (just to test)
for round_num in range(3):
    print(f"\n=== Round {round_num + 1} ===")
    game.play_round()
    game.show_game_state()



=== Round 1 ===

--- Ruud's turn ---
Ruud drew Queen of Clubs
Ruud discarded 9 of Hearts

--- Mariska's turn ---
Mariska drew Jack of Clubs
Mariska discarded King of Hearts

--- Menno's turn ---
Menno drew King of Diamonds
Menno discarded Ace of Spades

--- Lobke's turn ---
Lobke drew 9 of Spades
Lobke discarded 4 of Spades

=== Game State ===
Ruud: 6 of Hearts, 9 of Hearts, 5 of Hearts, King of Spades, Queen of Hearts, 5 of Spades, 7 of Hearts, 9 of Clubs, 6 of Spades, 7 of Spades, Queen of Clubs
Mariska: 10 of Spades, 6 of Diamonds, Ace of Spades, 6 of Diamonds, 10 of Clubs, 2 of Spades, 4 of Spades, 9 of Spades, Jack of Hearts, 7 of Spades, Jack of Clubs
Menno: King of Clubs, Jack of Clubs, 6 of Clubs, Jack of Diamonds, Queen of Clubs, 10 of Hearts, 5 of Clubs, Jack of Spades, 5 of Spades, 2 of Clubs, King of Diamonds
Lobke: Joker, 9 of Diamonds, 2 of Diamonds, Ace of Clubs, Ace of Clubs, 4 of Hearts, 7 of Hearts, 10 of Clubs, 2 of Hearts, 10 of Hearts, 9 of Spades
DiscardPile(top=